### Distributed lag modelling 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from matplotlib import style
sns.set(style='whitegrid',color_codes=True)

import warnings
warnings.filterwarnings("ignore")

import numpy.random as random
import statsmodels.api as sm
import statsmodels.tsa as tsa
import statsmodels.formula.api as smf

In [2]:
#Data preparation for CameraAccessory category
Master_Home=pd.read_csv('Master_Home.csv')

In [3]:
Master_Home.drop(["Unnamed: 0"], axis = 1, inplace = True)
Master_Home.head()

,Week,gmv_sum,units_sum,sla_mean,product_mrp_mean,product_mrp_sum,product_procurement_sla_mean,Weekly_Investment_Total_mean,Weekly_Investment_TV_mean,Weekly_Investment_Digital_mean,...,Weekly_Digital_adstock_mean,Weekly_Sponsorship_adstock_mean,Weekly_Online_Marketing_adstock_mean,Weekly_SEM_adstock_mean,Weekly_affiliates_adstock_mean,Sale_Flag,Rain_Flag,Snow_Flag,Max Temp (°C)_max,Min Temp (°C)_min
0,1,8.941600e+05,389.0,6.325581,3586.113695,1387826.0,2.824289,0.000000,0.050000,0.560000,...,0.550000,1.647615,0.294951,1.116377,0.121612,0.0,1,0,28.0,10.0
1,2,4.780028e+06,2260.0,6.829485,3318.915631,7474198.0,2.813499,0.000000,0.050000,0.560000,...,0.550000,1.647615,0.294951,1.116377,0.121612,0.0,1,0,29.0,12.5
2,3,5.002346e+06,2316.0,6.766522,3528.674348,8115951.0,2.794348,0.000000,0.050000,0.560000,...,0.550000,1.647615,0.294951,1.116377,0.121612,1.0,1,0,33.0,13.5
3,4,4.218112e+06,2095.0,6.836695,3311.164745,6893845.0,2.822286,0.000000,0.050000,0.560000,...,0.550000,1.647615,0.294951,1.116377,0.121612,0.0,1,0,33.5,14.5
4,5,2.002045e+06,946.0,7.063830,3483.221277,3274228.0,2.669149,0.001202,0.049947,0.559702,...,0.550011,1.646990,0.294824,1.116378,0.121565,0.0,1,0,33.5,15.0


In [4]:
Master_Home.columns

Index(['Week', 'gmv_sum', 'units_sum', 'sla_mean', 'product_mrp_mean',
       'product_mrp_sum', 'product_procurement_sla_mean',
       'Weekly_Investment_Total_mean', 'Weekly_Investment_TV_mean',
       'Weekly_Investment_Digital_mean', 'Weekly_Investment_Sponsorship_mean',
       'Weekly_Investment_Content_Marketing_mean',
       'Weekly_Investment_Online_Marketing_mean',
       'Weekly_Investment_affiliates_mean', 'Weekly_Investment_SEM_mean',
       'Weekly_Investment_Radio_mean', 'Weekly_Investment_Other_mean',
       'NPS_mean', 'Pay_Day_Flag', 's1_fact.order_payment_type_sum',
       'Prepaid_Flag_sum', 'Weekly_TV_adstock_mean',
       'Weekly_Digital_adstock_mean', 'Weekly_Sponsorship_adstock_mean',
       'Weekly_Online_Marketing_adstock_mean', 'Weekly_SEM_adstock_mean',
       'Weekly_affiliates_adstock_mean', 'Sale_Flag', 'Rain_Flag', 'Snow_Flag',
       'Max Temp (°C)_max', 'Min Temp (°C)_min'],
      dtype='object')

In [5]:
Master_Home.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 32 columns):
Week                                        50 non-null int64
gmv_sum                                     50 non-null float64
units_sum                                   50 non-null float64
sla_mean                                    50 non-null float64
product_mrp_mean                            50 non-null float64
product_mrp_sum                             50 non-null float64
product_procurement_sla_mean                50 non-null float64
Weekly_Investment_Total_mean                50 non-null float64
Weekly_Investment_TV_mean                   50 non-null float64
Weekly_Investment_Digital_mean              50 non-null float64
Weekly_Investment_Sponsorship_mean          50 non-null float64
Weekly_Investment_Content_Marketing_mean    50 non-null float64
Weekly_Investment_Online_Marketing_mean     50 non-null float64
Weekly_Investment_affiliates_mean           50 non-null float64
Wee

In [6]:
Master_Home.describe()

,Week,gmv_sum,units_sum,sla_mean,product_mrp_mean,product_mrp_sum,product_procurement_sla_mean,Weekly_Investment_Total_mean,Weekly_Investment_TV_mean,Weekly_Investment_Digital_mean,...,Weekly_Digital_adstock_mean,Weekly_Sponsorship_adstock_mean,Weekly_Online_Marketing_adstock_mean,Weekly_SEM_adstock_mean,Weekly_affiliates_adstock_mean,Sale_Flag,Rain_Flag,Snow_Flag,Max Temp (°C)_max,Min Temp (°C)_min
count,50.000000,5.000000e+01,50.000000,50.000000,50.000000,5.000000e+01,50.000000,50.000000,50.000000,50.000000,...,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.00000
mean,28.200000,1.972256e+06,2205.060000,5.609516,3663.208221,8.271842e+06,2.674271,16.501063,0.884590,0.569984,...,1.070361,13.441523,7.063279,3.324888,2.235837,0.260000,0.800000,0.160000,21.047000,3.24000
std,15.069905,3.229620e+06,1118.917378,0.689579,408.665948,5.296587e+06,0.134798,9.218751,0.535835,0.688148,...,0.744625,5.989645,2.825240,1.879364,0.895949,0.443087,0.404061,0.370328,9.175068,9.03465
min,1.000000,0.000000e+00,200.000000,3.775000,3119.285945,6.975550e+05,2.354839,0.000000,0.049947,0.100000,...,0.490000,1.646990,0.294824,1.116377,0.121565,0.000000,0.000000,0.000000,-5.000000,-18.00000
25%,16.250000,0.000000e+00,1799.250000,5.191102,3375.056261,5.906319e+06,2.574698,10.855000,0.370000,0.251658,...,0.560000,11.479901,6.404659,2.152843,2.044132,0.000000,1.000000,0.000000,15.625000,-3.87500
50%,28.500000,6.502655e+05,2090.500000,5.577786,3539.172040,7.112966e+06,2.663455,16.490000,0.940000,0.320000,...,0.870000,13.082098,8.021926,2.645456,2.693839,0.000000,1.000000,0.000000,22.000000,4.25000
75%,40.750000,3.815432e+06,2599.000000,6.000761,3954.490779,1.034505e+07,2.779271,22.020000,1.200000,0.559926,...,1.458639,15.754316,9.102428,4.779131,2.857837,0.750000,1.000000,0.000000,28.750000,11.87500
max,53.000000,1.908416e+07,7884.000000,7.063830,4671.770751,3.664070e+07,3.017100,37.810000,2.070000,2.800000,...,3.110000,26.057550,9.586060,8.063207,3.028860,1.000000,1.000000,1.000000,35.000000,18.00000


In [7]:
#Creating Promoting Pricing Effect

#Calculating percentage of discount offered.
Master_Home['discount_off_percent'] = ((Master_Home['product_mrp_mean']-Master_Home['gmv_sum'])/Master_Home['product_mrp_mean'])*100


In [8]:
Master_Home.head()

,Week,gmv_sum,units_sum,sla_mean,product_mrp_mean,product_mrp_sum,product_procurement_sla_mean,Weekly_Investment_Total_mean,Weekly_Investment_TV_mean,Weekly_Investment_Digital_mean,...,Weekly_Sponsorship_adstock_mean,Weekly_Online_Marketing_adstock_mean,Weekly_SEM_adstock_mean,Weekly_affiliates_adstock_mean,Sale_Flag,Rain_Flag,Snow_Flag,Max Temp (°C)_max,Min Temp (°C)_min,discount_off_percent
0,1,8.941600e+05,389.0,6.325581,3586.113695,1387826.0,2.824289,0.000000,0.050000,0.560000,...,1.647615,0.294951,1.116377,0.121612,0.0,1,0,28.0,10.0,-24833.955698
1,2,4.780028e+06,2260.0,6.829485,3318.915631,7474198.0,2.813499,0.000000,0.050000,0.560000,...,1.647615,0.294951,1.116377,0.121612,0.0,1,0,29.0,12.5,-143923.791659
2,3,5.002346e+06,2316.0,6.766522,3528.674348,8115951.0,2.794348,0.000000,0.050000,0.560000,...,1.647615,0.294951,1.116377,0.121612,1.0,1,0,33.0,13.5,-141662.755837
3,4,4.218112e+06,2095.0,6.836695,3311.164745,6893845.0,2.822286,0.000000,0.050000,0.560000,...,1.647615,0.294951,1.116377,0.121612,0.0,1,0,33.5,14.5,-127290.580786
4,5,2.002045e+06,946.0,7.063830,3483.221277,3274228.0,2.669149,0.001202,0.049947,0.559702,...,1.646990,0.294824,1.116378,0.121565,0.0,1,0,33.5,15.0,-57376.825071


In [9]:
#Calculating List Price for all skus
Master_Home['list_price'] = (Master_Home['gmv_sum']/Master_Home['units_sum'])

In [10]:
Master_Home.head()

,Week,gmv_sum,units_sum,sla_mean,product_mrp_mean,product_mrp_sum,product_procurement_sla_mean,Weekly_Investment_Total_mean,Weekly_Investment_TV_mean,Weekly_Investment_Digital_mean,...,Weekly_Online_Marketing_adstock_mean,Weekly_SEM_adstock_mean,Weekly_affiliates_adstock_mean,Sale_Flag,Rain_Flag,Snow_Flag,Max Temp (°C)_max,Min Temp (°C)_min,discount_off_percent,list_price
0,1,8.941600e+05,389.0,6.325581,3586.113695,1387826.0,2.824289,0.000000,0.050000,0.560000,...,0.294951,1.116377,0.121612,0.0,1,0,28.0,10.0,-24833.955698,2298.611825
1,2,4.780028e+06,2260.0,6.829485,3318.915631,7474198.0,2.813499,0.000000,0.050000,0.560000,...,0.294951,1.116377,0.121612,0.0,1,0,29.0,12.5,-143923.791659,2115.056696
2,3,5.002346e+06,2316.0,6.766522,3528.674348,8115951.0,2.794348,0.000000,0.050000,0.560000,...,0.294951,1.116377,0.121612,1.0,1,0,33.0,13.5,-141662.755837,2159.907599
3,4,4.218112e+06,2095.0,6.836695,3311.164745,6893845.0,2.822286,0.000000,0.050000,0.560000,...,0.294951,1.116377,0.121612,0.0,1,0,33.5,14.5,-127290.580786,2013.418616
4,5,2.002045e+06,946.0,7.063830,3483.221277,3274228.0,2.669149,0.001202,0.049947,0.559702,...,0.294824,1.116378,0.121565,0.0,1,0,33.5,15.0,-57376.825071,2116.326638


In [11]:
week_range = range(0,50,1)

In [12]:
#Calculating Shelf Price inflation
for i in week_range:
    Master_Home.loc[(Master_Home['Week'] == i+1), 'shelf_price_infl'] = Master_Home.iloc[i,33]/ Master_Home.iloc[i-1,33]

In [13]:
Master_Home.head()

,Week,gmv_sum,units_sum,sla_mean,product_mrp_mean,product_mrp_sum,product_procurement_sla_mean,Weekly_Investment_Total_mean,Weekly_Investment_TV_mean,Weekly_Investment_Digital_mean,...,Weekly_SEM_adstock_mean,Weekly_affiliates_adstock_mean,Sale_Flag,Rain_Flag,Snow_Flag,Max Temp (°C)_max,Min Temp (°C)_min,discount_off_percent,list_price,shelf_price_infl
0,1,8.941600e+05,389.0,6.325581,3586.113695,1387826.0,2.824289,0.000000,0.050000,0.560000,...,1.116377,0.121612,0.0,1,0,28.0,10.0,-24833.955698,2298.611825,16.164072
1,2,4.780028e+06,2260.0,6.829485,3318.915631,7474198.0,2.813499,0.000000,0.050000,0.560000,...,1.116377,0.121612,0.0,1,0,29.0,12.5,-143923.791659,2115.056696,0.920145
2,3,5.002346e+06,2316.0,6.766522,3528.674348,8115951.0,2.794348,0.000000,0.050000,0.560000,...,1.116377,0.121612,1.0,1,0,33.0,13.5,-141662.755837,2159.907599,1.021206
3,4,4.218112e+06,2095.0,6.836695,3311.164745,6893845.0,2.822286,0.000000,0.050000,0.560000,...,1.116377,0.121612,0.0,1,0,33.5,14.5,-127290.580786,2013.418616,0.932178
4,5,2.002045e+06,946.0,7.063830,3483.221277,3274228.0,2.669149,0.001202,0.049947,0.559702,...,1.116378,0.121565,0.0,1,0,33.5,15.0,-57376.825071,2116.326638,1.051111


In [14]:
Master_Home['shelf_price_infl'].fillna(0,inplace=True)
Master_Home['shelf_price_infl'].isna()

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
20    False
21    False
22    False
23    False
24    False
25    False
26    False
27    False
28    False
29    False
30    False
31    False
32    False
33    False
34    False
35    False
36    False
37    False
38    False
39    False
40    False
41    False
42    False
43    False
44    False
45    False
46    False
47    False
48    False
49    False
Name: shelf_price_infl, dtype: bool

In [15]:
Master_Home.loc[(Master_Home['shelf_price_infl'] == np.inf), 'shelf_price_infl'] = 0.0

In [16]:
Master_Home['shelf_price_infl']

0     16.164072
1      0.920145
2      1.021206
3      0.932178
4      1.051111
5      0.943199
6      1.018946
7      1.011137
8      0.959268
9      1.313128
10     0.939794
11     0.912475
12     1.045163
13     0.880018
14     1.088461
15     0.775462
16     0.000000
17     0.000000
18     0.000000
19     0.000000
20     0.000000
21     1.077161
22     1.360017
23     1.032482
24     1.063056
25     0.821431
26     1.394501
27     0.651439
28     1.350681
29     0.000000
30     0.000000
31     0.000000
32     0.000000
33     0.000000
34     0.000000
35     0.000000
36     0.000000
37     0.000000
38     0.000000
39     0.000000
40     0.000000
41     0.000000
42     0.000000
43     1.225626
44     0.626430
45     0.940533
46     1.002365
47     0.000000
48     0.000000
49     0.000000
Name: shelf_price_infl, dtype: float64

In [17]:
Master_Home.head()

,Week,gmv_sum,units_sum,sla_mean,product_mrp_mean,product_mrp_sum,product_procurement_sla_mean,Weekly_Investment_Total_mean,Weekly_Investment_TV_mean,Weekly_Investment_Digital_mean,...,Weekly_SEM_adstock_mean,Weekly_affiliates_adstock_mean,Sale_Flag,Rain_Flag,Snow_Flag,Max Temp (°C)_max,Min Temp (°C)_min,discount_off_percent,list_price,shelf_price_infl
0,1,8.941600e+05,389.0,6.325581,3586.113695,1387826.0,2.824289,0.000000,0.050000,0.560000,...,1.116377,0.121612,0.0,1,0,28.0,10.0,-24833.955698,2298.611825,16.164072
1,2,4.780028e+06,2260.0,6.829485,3318.915631,7474198.0,2.813499,0.000000,0.050000,0.560000,...,1.116377,0.121612,0.0,1,0,29.0,12.5,-143923.791659,2115.056696,0.920145
2,3,5.002346e+06,2316.0,6.766522,3528.674348,8115951.0,2.794348,0.000000,0.050000,0.560000,...,1.116377,0.121612,1.0,1,0,33.0,13.5,-141662.755837,2159.907599,1.021206
3,4,4.218112e+06,2095.0,6.836695,3311.164745,6893845.0,2.822286,0.000000,0.050000,0.560000,...,1.116377,0.121612,0.0,1,0,33.5,14.5,-127290.580786,2013.418616,0.932178
4,5,2.002045e+06,946.0,7.063830,3483.221277,3274228.0,2.669149,0.001202,0.049947,0.559702,...,1.116378,0.121565,0.0,1,0,33.5,15.0,-57376.825071,2116.326638,1.051111


In [18]:
from sklearn.model_selection import train_test_split

# We specify this so that the train and test data set always have the same rows, respectively
np.random.seed(0)
Home_train, Home_test = train_test_split(Master_Home, train_size = 0.7, test_size = 0.3, random_state = 100)

In [19]:
#Implementing Distributed Lag Model with LinearRegression for CameraAccessory on training dataset
Home_y_train = Home_train.pop('gmv_sum')
Home_X_train = Home_train

In [20]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
def find_vif(X_train):
    vif = pd.DataFrame()
    vif['Features'] = X_train.columns
    vif['VIF'] = [variance_inflation_factor(X_train.values, i) for i in range(X_train.shape[1])]
    vif['VIF'] = round(vif['VIF'], 2)
    vif = vif.sort_values(by = "VIF", ascending = False)
    return vif

In [21]:
home_vif = find_vif(Home_X_train)

In [22]:
home_vif

,Features,VIF
14,Weekly_Investment_Radio_mean,inf
10,Weekly_Investment_Content_Marketing_mean,inf
20,Weekly_TV_adstock_mean,inf
16,NPS_mean,inf
15,Weekly_Investment_Other_mean,inf
23,Weekly_Online_Marketing_adstock_mean,inf
13,Weekly_Investment_SEM_mean,inf
12,Weekly_Investment_affiliates_mean,inf
22,Weekly_Sponsorship_adstock_mean,inf
11,Weekly_Investment_Online_Marketing_mean,inf


In [23]:
# Dropping columns with high VIF values 

Home_X_train.drop(['NPS_mean','Weekly_Investment_Other_mean','Weekly_Online_Marketing_adstock_mean','Weekly_Investment_SEM_mean','Weekly_Investment_affiliates_mean','Weekly_Sponsorship_adstock_mean','Weekly_Investment_Online_Marketing_mean','Weekly_Investment_Sponsorship_mean','Weekly_Investment_Digital_mean','Weekly_Investment_TV_mean','Weekly_Investment_Total_mean','Weekly_SEM_adstock_mean','Weekly_affiliates_adstock_mean','Weekly_Digital_adstock_mean'],axis=1,inplace = True)

In [24]:
home_vif = find_vif(Home_X_train)
home_vif

,Features,VIF
1,units_sum,598494.57
9,s1_fact.order_payment_type_sum,361568.88
10,Prepaid_Flag_sum,34413.33
3,product_mrp_mean,1457.97
5,product_procurement_sla_mean,1100.19
4,product_mrp_sum,1076.04
2,sla_mean,214.37
18,list_price,61.95
17,discount_off_percent,58.56
15,Max Temp (°C)_max,51.51


In [25]:
# Dropping columns with high VIF values 

Home_X_train.drop(['units_sum','s1_fact.order_payment_type_sum','Prepaid_Flag_sum'],axis=1,inplace = True)

In [26]:
home_vif = find_vif(Home_X_train)
home_vif

,Features,VIF
4,product_procurement_sla_mean,680.40
2,product_mrp_mean,545.11
1,sla_mean,167.21
15,list_price,51.78
3,product_mrp_sum,49.28
12,Max Temp (°C)_max,48.32
14,discount_off_percent,46.06
0,Week,30.54
8,Weekly_TV_adstock_mean,20.22
10,Rain_Flag,13.71


In [27]:
# Dropping columns with high VIF values 

Home_X_train.drop(['product_procurement_sla_mean','product_mrp_mean'],axis=1,inplace = True)
home_vif = find_vif(Home_X_train)
home_vif

,Features,VIF
1,sla_mean,57.15
10,Max Temp (°C)_max,46.16
13,list_price,45.33
12,discount_off_percent,38.05
2,product_mrp_sum,28.64
0,Week,26.30
6,Weekly_TV_adstock_mean,18.34
11,Min Temp (°C)_min,11.98
8,Rain_Flag,10.14
14,shelf_price_infl,6.74


In [28]:
import statsmodels.api as sm
def returnOLS(y_train,x_train):
    # Add a constant
    X_lm = sm.add_constant(x_train)
    # Create a first fitted model
    lr = sm.OLS(y_train,X_lm).fit()
    y_pred = lr.predict(X_lm)
    return lr,y_pred

In [29]:
home_lr1,lr1_pred = returnOLS(Home_y_train, Home_X_train)
print(home_lr1.summary())

                            OLS Regression Results                            
Dep. Variable:                gmv_sum   R-squared:                       0.995
Model:                            OLS   Adj. R-squared:                  0.991
Method:                 Least Squares   F-statistic:                     245.8
Date:                Mon, 16 Sep 2019   Prob (F-statistic):           1.68e-18
Time:                        20:54:19   Log-Likelihood:                -485.79
No. Observations:                  35   AIC:                             1004.
Df Residuals:                      19   BIC:                             1028.
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                                               coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------

In [30]:
#Eliminating variables where p-values>0.05

Home_X_train.drop(['product_mrp_sum','list_price'],axis=1,inplace = True)
home_lr2,lr2_pred = returnOLS(Home_y_train, Home_X_train)
print(home_lr2.summary())

                            OLS Regression Results                            
Dep. Variable:                gmv_sum   R-squared:                       0.991
Model:                            OLS   Adj. R-squared:                  0.985
Method:                 Least Squares   F-statistic:                     170.4
Date:                Mon, 16 Sep 2019   Prob (F-statistic):           3.01e-18
Time:                        20:54:19   Log-Likelihood:                -496.38
No. Observations:                  35   AIC:                             1021.
Df Residuals:                      21   BIC:                             1043.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                                               coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------

In [31]:
#3rd Iteration : Eliminating variables where p-values>0.05
Home_X_train.drop(['Max Temp (°C)_max','Snow_Flag','Weekly_Investment_Content_Marketing_mean'],axis=1,inplace = True)
home_lr3,lr3_pred = returnOLS(Home_y_train, Home_X_train)
print(home_lr3.summary())

                            OLS Regression Results                            
Dep. Variable:                gmv_sum   R-squared:                       0.990
Model:                            OLS   Adj. R-squared:                  0.986
Method:                 Least Squares   F-statistic:                     243.8
Date:                Mon, 16 Sep 2019   Prob (F-statistic):           1.30e-21
Time:                        20:54:19   Log-Likelihood:                -497.04
No. Observations:                  35   AIC:                             1016.
Df Residuals:                      24   BIC:                             1033.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
const           

In [32]:
#3rd Iteration : Eliminating variables where p-values>0.05
Home_X_train.drop(['Min Temp (°C)_min','Weekly_Investment_Radio_mean'],axis=1,inplace = True)
home_lr4,lr4_pred = returnOLS(Home_y_train, Home_X_train)
print(home_lr4.summary())

                            OLS Regression Results                            
Dep. Variable:                gmv_sum   R-squared:                       0.989
Model:                            OLS   Adj. R-squared:                  0.986
Method:                 Least Squares   F-statistic:                     296.8
Date:                Mon, 16 Sep 2019   Prob (F-statistic):           1.53e-23
Time:                        20:54:19   Log-Likelihood:                -498.88
No. Observations:                  35   AIC:                             1016.
Df Residuals:                      26   BIC:                             1030.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                  -5.78

In [33]:
#5th Iteration : Eliminating variables where p-values>0.05

Home_X_train.drop(['Pay_Day_Flag','Rain_Flag'],axis=1,inplace = True)
home_lr5,lr5_pred = returnOLS(Home_y_train, Home_X_train)
print(home_lr5.summary())

                            OLS Regression Results                            
Dep. Variable:                gmv_sum   R-squared:                       0.989
Model:                            OLS   Adj. R-squared:                  0.987
Method:                 Least Squares   F-statistic:                     421.3
Date:                Mon, 16 Sep 2019   Prob (F-statistic):           4.22e-26
Time:                        20:54:20   Log-Likelihood:                -499.08
No. Observations:                  35   AIC:                             1012.
Df Residuals:                      28   BIC:                             1023.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                  -3.92

In [34]:
#6th Iteration : Eliminating variables where p-values>0.05
Home_X_train.drop(['Sale_Flag'],axis=1,inplace = True)
home_lr6,lr6_pred = returnOLS(Home_y_train, Home_X_train)
print(home_lr6.summary())

                            OLS Regression Results                            
Dep. Variable:                gmv_sum   R-squared:                       0.989
Model:                            OLS   Adj. R-squared:                  0.987
Method:                 Least Squares   F-statistic:                     501.8
Date:                Mon, 16 Sep 2019   Prob (F-statistic):           3.19e-27
Time:                        20:54:20   Log-Likelihood:                -499.81
No. Observations:                  35   AIC:                             1012.
Df Residuals:                      29   BIC:                             1021.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                   1.73

In [35]:
#6th Iteration : Eliminating variables where p-values>0.05

Home_X_train.drop(['Week'],axis=1,inplace = True)
home_lr7,lr7_pred = returnOLS(Home_y_train, Home_X_train)
print(home_lr7.summary())

                            OLS Regression Results                            
Dep. Variable:                gmv_sum   R-squared:                       0.988
Model:                            OLS   Adj. R-squared:                  0.986
Method:                 Least Squares   F-statistic:                     596.0
Date:                Mon, 16 Sep 2019   Prob (F-statistic):           4.12e-28
Time:                        20:54:20   Log-Likelihood:                -501.28
No. Observations:                  35   AIC:                             1013.
Df Residuals:                      30   BIC:                             1020.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                   1.61

In [36]:
#8th Iteration : Eliminating variables where p-values>0.05

Home_X_train.drop(['shelf_price_infl'],axis=1,inplace = True)
home_lr8,lr8_pred = returnOLS(Home_y_train, Home_X_train)
print(home_lr8.summary())

                            OLS Regression Results                            
Dep. Variable:                gmv_sum   R-squared:                       0.987
Model:                            OLS   Adj. R-squared:                  0.986
Method:                 Least Squares   F-statistic:                     815.6
Date:                Mon, 16 Sep 2019   Prob (F-statistic):           1.46e-29
Time:                        20:54:20   Log-Likelihood:                -501.40
No. Observations:                  35   AIC:                             1011.
Df Residuals:                      31   BIC:                             1017.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                   1.64

In [37]:
find_vif(Home_X_train)

,Features,VIF
0,sla_mean,6.61
1,Weekly_TV_adstock_mean,4.18
2,discount_off_percent,2.45


In [38]:
# RMSE and R2_Score Calculation
from sklearn.metrics import mean_squared_error, r2_score
def rmse_r2score(y_test,y_pred):
    rmse = (np.sqrt(mean_squared_error(y_test, y_pred)))
    r2score = r2_score(y_test, y_pred)
    return rmse, r2score

In [39]:
Home_y_test = Home_test.pop('gmv_sum')
Home_X_test = Home_test[Home_X_train.columns]
#Add a constant
Home_X_test_lm = sm.add_constant(Home_X_test)
# Create a first fitted model
y_pred = home_lr8.predict(Home_X_test_lm)
home_rmse,home_r2score = rmse_r2score(Home_y_test,y_pred)

In [40]:
print("RMSE", home_rmse,"R2 ",home_r2score)

RMSE 523877.42595945543 R2  0.88371092219538


In [41]:
#Data preparation for CameraAccessory category
Master_Gaming=pd.read_csv('Master_Gaming.csv')

In [42]:
Master_Gaming.drop(["Unnamed: 0"], axis = 1, inplace = True)
Master_Gaming.head()

,Week,gmv_sum,units_sum,sla_mean,product_mrp_mean,product_mrp_sum,product_procurement_sla_mean,Weekly_Investment_Total_mean,Weekly_Investment_TV_mean,Weekly_Investment_Digital_mean,...,Weekly_Digital_adstock_mean,Weekly_Sponsorship_adstock_mean,Weekly_Online_Marketing_adstock_mean,Weekly_SEM_adstock_mean,Weekly_affiliates_adstock_mean,Sale_Flag,Rain_Flag,Snow_Flag,Max Temp (°C)_max,Min Temp (°C)_min
0,1,7.397400e+05,885,4.212963,1451.935185,1254472.0,3.509259,0.000000,0.050000,0.560000,...,0.550000,1.647615,0.294951,1.116377,0.121612,0.0,1,0,28.0,10.0
1,2,2.442736e+06,2619,5.983991,1469.899649,3764413.0,2.843811,0.000000,0.050000,0.560000,...,0.550000,1.647615,0.294951,1.116377,0.121612,0.0,1,0,29.0,12.5
2,3,2.245456e+06,2754,5.175898,1324.206213,3538279.0,2.609281,0.000000,0.050000,0.560000,...,0.550000,1.647615,0.294951,1.116377,0.121612,1.0,1,0,33.0,13.5
3,4,2.570502e+06,2430,5.433673,1834.422194,4314561.0,2.359694,0.000000,0.050000,0.560000,...,0.550000,1.647615,0.294951,1.116377,0.121612,0.0,1,0,33.5,14.5
4,5,1.014782e+06,993,5.909751,1798.849585,1734091.0,2.438797,0.002344,0.049896,0.559419,...,0.550021,1.646397,0.294704,1.116378,0.121520,0.0,1,0,33.5,15.0


In [43]:
#Creating Promoting Pricing Effect

#Calculating percentage of discount offered.
Master_Gaming['discount_off_percent'] = ((Master_Gaming['product_mrp_mean']-Master_Gaming['gmv_sum'])/Master_Gaming['product_mrp_mean'])*100


In [44]:
#Calculating List Price for all skus
Master_Gaming['list_price'] = (Master_Gaming['gmv_sum']/Master_Gaming['units_sum'])

In [45]:
week_range = range(0,52,1)

In [46]:
#Calculating Shelf Price inflation
for i in week_range:
    Master_Gaming.loc[(Master_Gaming['Week'] == i+1), 'shelf_price_infl'] = Master_Gaming.iloc[i,33]/ Master_Gaming.iloc[i-1,33]

In [47]:
Master_Gaming.head()

,Week,gmv_sum,units_sum,sla_mean,product_mrp_mean,product_mrp_sum,product_procurement_sla_mean,Weekly_Investment_Total_mean,Weekly_Investment_TV_mean,Weekly_Investment_Digital_mean,...,Weekly_SEM_adstock_mean,Weekly_affiliates_adstock_mean,Sale_Flag,Rain_Flag,Snow_Flag,Max Temp (°C)_max,Min Temp (°C)_min,discount_off_percent,list_price,shelf_price_infl
0,1,7.397400e+05,885,4.212963,1451.935185,1254472.0,3.509259,0.000000,0.050000,0.560000,...,1.116377,0.121612,0.0,1,0,28.0,10.0,-50848.555249,835.864407,inf
1,2,2.442736e+06,2619,5.983991,1469.899649,3764413.0,2.843811,0.000000,0.050000,0.560000,...,1.116377,0.121612,0.0,1,0,29.0,12.5,-166083.861760,932.697976,1.115848
2,3,2.245456e+06,2754,5.175898,1324.206213,3538279.0,2.609281,0.000000,0.050000,0.560000,...,1.116377,0.121612,1.0,1,0,33.0,13.5,-169469.991280,815.343631,0.874178
3,4,2.570502e+06,2430,5.433673,1834.422194,4314561.0,2.359694,0.000000,0.050000,0.560000,...,1.116377,0.121612,0.0,1,0,33.5,14.5,-140026.003086,1057.819959,1.297392
4,5,1.014782e+06,993,5.909751,1798.849585,1734091.0,2.438797,0.002344,0.049896,0.559419,...,1.116378,0.121520,0.0,1,0,33.5,15.0,-56312.832314,1021.935549,0.966077


In [48]:
Master_Gaming['shelf_price_infl'].fillna(0,inplace=True)
Master_Gaming['shelf_price_infl'].isna()

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
20    False
21    False
22    False
23    False
24    False
25    False
26    False
27    False
28    False
29    False
30    False
31    False
32    False
33    False
34    False
35    False
36    False
37    False
38    False
39    False
40    False
41    False
42    False
43    False
44    False
45    False
46    False
47    False
48    False
49    False
50    False
51    False
52    False
Name: shelf_price_infl, dtype: bool

In [49]:
Master_Gaming.loc[(Master_Gaming['shelf_price_infl'] == np.inf), 'shelf_price_infl'] = 0.0

In [50]:
Master_Gaming.head()

,Week,gmv_sum,units_sum,sla_mean,product_mrp_mean,product_mrp_sum,product_procurement_sla_mean,Weekly_Investment_Total_mean,Weekly_Investment_TV_mean,Weekly_Investment_Digital_mean,...,Weekly_SEM_adstock_mean,Weekly_affiliates_adstock_mean,Sale_Flag,Rain_Flag,Snow_Flag,Max Temp (°C)_max,Min Temp (°C)_min,discount_off_percent,list_price,shelf_price_infl
0,1,7.397400e+05,885,4.212963,1451.935185,1254472.0,3.509259,0.000000,0.050000,0.560000,...,1.116377,0.121612,0.0,1,0,28.0,10.0,-50848.555249,835.864407,0.000000
1,2,2.442736e+06,2619,5.983991,1469.899649,3764413.0,2.843811,0.000000,0.050000,0.560000,...,1.116377,0.121612,0.0,1,0,29.0,12.5,-166083.861760,932.697976,1.115848
2,3,2.245456e+06,2754,5.175898,1324.206213,3538279.0,2.609281,0.000000,0.050000,0.560000,...,1.116377,0.121612,1.0,1,0,33.0,13.5,-169469.991280,815.343631,0.874178
3,4,2.570502e+06,2430,5.433673,1834.422194,4314561.0,2.359694,0.000000,0.050000,0.560000,...,1.116377,0.121612,0.0,1,0,33.5,14.5,-140026.003086,1057.819959,1.297392
4,5,1.014782e+06,993,5.909751,1798.849585,1734091.0,2.438797,0.002344,0.049896,0.559419,...,1.116378,0.121520,0.0,1,0,33.5,15.0,-56312.832314,1021.935549,0.966077


In [51]:
from sklearn.model_selection import train_test_split

# We specify this so that the train and test data set always have the same rows, respectively
np.random.seed(0)
Gaming_train, Gaming_test = train_test_split(Master_Gaming, train_size = 0.7, test_size = 0.3, random_state = 100)

In [52]:
#Implementing Distributed Lag Model with LinearRegression for CameraAccessory on training dataset
Gaming_y_train = Gaming_train.pop('gmv_sum')
Gaming_X_train = Gaming_train

In [53]:
#Determining VIF
home_vif = find_vif(Gaming_X_train)
home_vif

,Features,VIF
14,Weekly_Investment_Radio_mean,inf
9,Weekly_Investment_Sponsorship_mean,inf
16,NPS_mean,inf
15,Weekly_Investment_Other_mean,inf
22,Weekly_Sponsorship_adstock_mean,inf
13,Weekly_Investment_SEM_mean,inf
12,Weekly_Investment_affiliates_mean,inf
11,Weekly_Investment_Online_Marketing_mean,inf
21,Weekly_Digital_adstock_mean,inf
10,Weekly_Investment_Content_Marketing_mean,inf


In [54]:
# Dropping columns with high VIF values 
Gaming_X_train.drop(['NPS_mean','Weekly_Investment_Other_mean','Weekly_Online_Marketing_adstock_mean','Weekly_Investment_SEM_mean','Weekly_Investment_affiliates_mean','Weekly_Sponsorship_adstock_mean','Weekly_Investment_Online_Marketing_mean','Weekly_Investment_Sponsorship_mean','Weekly_Investment_Digital_mean','Weekly_Investment_TV_mean','Weekly_Investment_Total_mean','Weekly_SEM_adstock_mean','Weekly_affiliates_adstock_mean','Weekly_Digital_adstock_mean'],axis=1,inplace = True)
#Determining VIF
Gaming_vif = find_vif(Gaming_X_train)
Gaming_vif

,Features,VIF
1,units_sum,216368.65
9,s1_fact.order_payment_type_sum,122536.87
10,Prepaid_Flag_sum,21141.62
4,product_mrp_sum,1358.55
2,sla_mean,529.33
3,product_mrp_mean,256.90
5,product_procurement_sla_mean,240.81
18,list_price,153.05
17,discount_off_percent,87.17
0,Week,65.42


In [55]:
# Dropping columns with high VIF values 

Gaming_X_train.drop(['units_sum','s1_fact.order_payment_type_sum','Prepaid_Flag_sum'],axis=1,inplace = True)
#Determining VIF
Gaming_vif = find_vif(Gaming_X_train)
Gaming_vif

,Features,VIF
1,sla_mean,345.07
2,product_mrp_mean,203.20
4,product_procurement_sla_mean,181.21
15,list_price,65.08
12,Max Temp (°C)_max,58.70
0,Week,53.15
8,Weekly_TV_adstock_mean,32.97
3,product_mrp_sum,27.15
13,Min Temp (°C)_min,16.01
14,discount_off_percent,14.53


In [56]:
# Dropping columns with high VIF values 

Gaming_X_train.drop(['sla_mean','product_mrp_mean','product_procurement_sla_mean'],axis=1,inplace = True)
#Determining VIF
Gaming_vif = find_vif(Gaming_X_train)
Gaming_vif

,Features,VIF
0,Week,36.22
9,Max Temp (°C)_max,30.55
12,list_price,25.42
5,Weekly_TV_adstock_mean,21.09
10,Min Temp (°C)_min,14.32
1,product_mrp_sum,12.40
11,discount_off_percent,8.23
7,Rain_Flag,7.51
13,shelf_price_infl,6.70
3,Weekly_Investment_Radio_mean,4.64


In [57]:
# Dropping columns with high VIF values 

Gaming_X_train.drop(['Week','Max Temp (°C)_max'],axis=1,inplace = True)
#Determining VIF
Gaming_vif = find_vif(Gaming_X_train)
Gaming_vif

,Features,VIF
10,list_price,13.66
0,product_mrp_sum,10.85
4,Weekly_TV_adstock_mean,8.88
9,discount_off_percent,7.82
6,Rain_Flag,7.10
11,shelf_price_infl,6.24
8,Min Temp (°C)_min,5.93
2,Weekly_Investment_Radio_mean,3.25
1,Weekly_Investment_Content_Marketing_mean,2.75
3,Pay_Day_Flag,2.58


In [58]:
# Dropping columns with high VIF values 

Gaming_X_train.drop(['list_price'],axis=1,inplace = True)
#Determining VIF
Gaming_vif = find_vif(Gaming_X_train)
Gaming_vif

,Features,VIF
0,product_mrp_sum,9.68
4,Weekly_TV_adstock_mean,8.82
6,Rain_Flag,6.85
9,discount_off_percent,4.93
8,Min Temp (°C)_min,4.76
10,shelf_price_infl,3.59
2,Weekly_Investment_Radio_mean,2.91
1,Weekly_Investment_Content_Marketing_mean,2.74
3,Pay_Day_Flag,2.54
5,Sale_Flag,1.92


In [59]:
# Dropping columns with high VIF values 

Gaming_X_train.drop(['product_mrp_sum','Weekly_TV_adstock_mean'],axis=1,inplace = True)
#Determining VIF
Gaming_vif = find_vif(Gaming_X_train)
Gaming_vif

,Features,VIF
4,Rain_Flag,4.06
6,Min Temp (°C)_min,3.67
7,discount_off_percent,3.37
8,shelf_price_infl,3.04
1,Weekly_Investment_Radio_mean,2.45
0,Weekly_Investment_Content_Marketing_mean,2.37
2,Pay_Day_Flag,2.22
3,Sale_Flag,1.80
5,Snow_Flag,1.77


In [60]:
Gaming_lr1,lr1_pred = returnOLS(Gaming_y_train,Gaming_X_train)
print(Gaming_lr1.summary())

                            OLS Regression Results                            
Dep. Variable:                gmv_sum   R-squared:                       0.989
Model:                            OLS   Adj. R-squared:                  0.985
Method:                 Least Squares   F-statistic:                     260.3
Date:                Mon, 16 Sep 2019   Prob (F-statistic):           7.42e-24
Time:                        20:54:24   Log-Likelihood:                -493.37
No. Observations:                  37   AIC:                             1007.
Df Residuals:                      27   BIC:                             1023.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                                               coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------

In [61]:
Gaming_X_train.drop(['Snow_Flag','Rain_Flag','Min Temp (°C)_min'],axis=1,inplace = True)
Gaming_lr2,lr2_pred = returnOLS(Gaming_y_train,Gaming_X_train)
print(Gaming_lr2.summary())

                            OLS Regression Results                            
Dep. Variable:                gmv_sum   R-squared:                       0.988
Model:                            OLS   Adj. R-squared:                  0.985
Method:                 Least Squares   F-statistic:                     399.3
Date:                Mon, 16 Sep 2019   Prob (F-statistic):           3.22e-27
Time:                        20:54:24   Log-Likelihood:                -494.89
No. Observations:                  37   AIC:                             1004.
Df Residuals:                      30   BIC:                             1015.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                                               coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------

In [62]:
Gaming_X_train.drop(['Sale_Flag','Weekly_Investment_Content_Marketing_mean'],axis=1,inplace = True)
Gaming_lr3,lr3_pred = returnOLS(Gaming_y_train,Gaming_X_train)
print(Gaming_lr3.summary())

                            OLS Regression Results                            
Dep. Variable:                gmv_sum   R-squared:                       0.986
Model:                            OLS   Adj. R-squared:                  0.984
Method:                 Least Squares   F-statistic:                     562.5
Date:                Mon, 16 Sep 2019   Prob (F-statistic):           3.75e-29
Time:                        20:54:24   Log-Likelihood:                -497.21
No. Observations:                  37   AIC:                             1004.
Df Residuals:                      32   BIC:                             1012.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
const           

In [63]:
Gaming_X_train.drop(['Weekly_Investment_Radio_mean'],axis=1,inplace = True)
Gaming_lr3,lr3_pred = returnOLS(Gaming_y_train,Gaming_X_train)
print(Gaming_lr3.summary())

                            OLS Regression Results                            
Dep. Variable:                gmv_sum   R-squared:                       0.985
Model:                            OLS   Adj. R-squared:                  0.983
Method:                 Least Squares   F-statistic:                     705.4
Date:                Mon, 16 Sep 2019   Prob (F-statistic):           5.50e-30
Time:                        20:54:24   Log-Likelihood:                -498.89
No. Observations:                  37   AIC:                             1006.
Df Residuals:                      33   BIC:                             1012.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                 8.845e+04 

In [89]:
Gaming_y_test = Gaming_test.pop('gmv_sum')
Gaming_X_test = Gaming_test[Gaming_X_train.columns]

KeyError: 'gmv_sum'

In [88]:
# Add a constant
Gaming_X_test_lm = sm.add_constant(Gaming_X_test)
# Create a first fitted model
y_pred = Gaming_lr3.predict(Gaming_X_test_lm)
Gaming_rmse,Gaming_r2score = rmse_r2score(Gaming_y_test,y_pred)

In [92]:
print('RMSE', Gaming_rmse, 'R2 ',Gaming_r2score)

RMSE 132187.99125604448 R2  0.9834161092624667


In [98]:
#Data preparation for CameraAccessory category
Master_Camera=pd.read_csv('Master_Camera.csv')

In [99]:
Master_Camera.drop(["Unnamed: 0"], axis = 1, inplace = True)
Master_Camera.head()

,Week,gmv_sum,units_sum,sla_mean,product_mrp_mean,product_mrp_sum,product_procurement_sla_mean,Weekly_Investment_Total_mean,Weekly_Investment_TV_mean,Weekly_Investment_Digital_mean,...,Weekly_Digital_adstock_mean,Weekly_Sponsorship_adstock_mean,Weekly_Online_Marketing_adstock_mean,Weekly_SEM_adstock_mean,Weekly_affiliates_adstock_mean,Sale_Flag,Rain_Flag,Snow_Flag,Max Temp (°C)_max,Min Temp (°C)_min
0,1,687143.0,523.0,5.805825,2492.768932,1283776.0,2.699029,0.000000,0.050000,0.560000,...,0.550000,1.647615,0.294951,1.116377,0.121612,0.0,1,0,28.0,10.0
1,2,3449874.0,3058.0,6.793416,2202.988579,6558297.0,2.657709,0.000000,0.050000,0.560000,...,0.550000,1.647615,0.294951,1.116377,0.121612,0.0,1,0,29.0,12.5
2,3,3834486.0,3060.0,6.116667,2317.125667,6951377.0,2.734000,0.000000,0.050000,0.560000,...,0.550000,1.647615,0.294951,1.116377,0.121612,1.0,1,0,33.0,13.5
3,4,3359335.0,2902.0,6.294076,2141.853667,6074297.0,2.722496,0.000000,0.050000,0.560000,...,0.550000,1.647615,0.294951,1.116377,0.121612,0.0,1,0,33.5,14.5
4,5,1443658.0,1226.0,6.373434,2148.959900,2572305.0,2.739348,0.001888,0.049916,0.559532,...,0.550017,1.646634,0.294752,1.116378,0.121538,0.0,1,0,33.5,15.0


In [100]:
#Creating Promoting Pricing Effect

#Calculating percentage of discount offered.
Master_Camera['discount_off_percent'] = ((Master_Camera['product_mrp_mean']-Master_Camera['gmv_sum'])/Master_Camera['product_mrp_mean'])*100


In [101]:
#Calculating List Price for all skus
Master_Camera['list_price'] = (Master_Camera['gmv_sum']/Master_Camera['units_sum'])

In [102]:
week_range = range(0,52,1)

In [103]:
#Calculating Shelf Price inflation
for i in week_range:
    Master_Camera.loc[(Master_Camera['Week'] == i+1), 'shelf_price_infl'] = Master_Camera.iloc[i,33]/ Master_Camera.iloc[i-1,33]

In [104]:
Master_Camera['shelf_price_infl'].fillna(0,inplace=True)
Master_Camera['shelf_price_infl'].isna()

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
20    False
21    False
22    False
23    False
24    False
25    False
26    False
27    False
28    False
29    False
30    False
31    False
32    False
33    False
34    False
35    False
36    False
37    False
38    False
39    False
40    False
41    False
42    False
43    False
44    False
45    False
46    False
47    False
48    False
49    False
50    False
51    False
Name: shelf_price_infl, dtype: bool

In [105]:
Master_Camera.loc[(Master_Camera['shelf_price_infl'] == np.inf), 'shelf_price_infl'] = 0.0

In [106]:
Master_Camera.head()

,Week,gmv_sum,units_sum,sla_mean,product_mrp_mean,product_mrp_sum,product_procurement_sla_mean,Weekly_Investment_Total_mean,Weekly_Investment_TV_mean,Weekly_Investment_Digital_mean,...,Weekly_SEM_adstock_mean,Weekly_affiliates_adstock_mean,Sale_Flag,Rain_Flag,Snow_Flag,Max Temp (°C)_max,Min Temp (°C)_min,discount_off_percent,list_price,shelf_price_infl
0,1,687143.0,523.0,5.805825,2492.768932,1283776.0,2.699029,0.000000,0.050000,0.560000,...,1.116377,0.121612,0.0,1,0,28.0,10.0,-27465.451060,1313.848948,62.309250
1,2,3449874.0,3058.0,6.793416,2202.988579,6558297.0,2.657709,0.000000,0.050000,0.560000,...,1.116377,0.121612,0.0,1,0,29.0,12.5,-156499.722428,1128.147155,0.858658
2,3,3834486.0,3060.0,6.116667,2317.125667,6951377.0,2.734000,0.000000,0.050000,0.560000,...,1.116377,0.121612,1.0,1,0,33.0,13.5,-165384.593916,1253.100000,1.110759
3,4,3359335.0,2902.0,6.294076,2141.853667,6074297.0,2.722496,0.000000,0.050000,0.560000,...,1.116377,0.121612,0.0,1,0,33.5,14.5,-156742.414192,1157.593039,0.923783
4,5,1443658.0,1226.0,6.373434,2148.959900,2572305.0,2.739348,0.001888,0.049916,0.559532,...,1.116378,0.121538,0.0,1,0,33.5,15.0,-67079.382927,1177.535073,1.017227


In [107]:
from sklearn.model_selection import train_test_split

# We specify this so that the train and test data set always have the same rows, respectively
np.random.seed(0)
Camera_train, Camera_test = train_test_split(Master_Camera, train_size = 0.7, test_size = 0.3, random_state = 100)

In [108]:
#Implementing Distributed Lag Model with LinearRegression for CameraAccessory on training dataset
Camera_y_train = Camera_train.pop('gmv_sum')
Camera_X_train = Camera_train

In [109]:
#Determining VIF
camera_vif = find_vif(Camera_X_train)
camera_vif

,Features,VIF
13,Weekly_Investment_SEM_mean,inf
11,Weekly_Investment_Online_Marketing_mean,inf
24,Weekly_SEM_adstock_mean,inf
23,Weekly_Online_Marketing_adstock_mean,inf
22,Weekly_Sponsorship_adstock_mean,inf
21,Weekly_Digital_adstock_mean,inf
20,Weekly_TV_adstock_mean,inf
16,NPS_mean,inf
15,Weekly_Investment_Other_mean,inf
14,Weekly_Investment_Radio_mean,inf


In [110]:
# Dropping columns with high VIF values 
Camera_X_train.drop(['Week','NPS_mean','Weekly_Investment_Other_mean','Weekly_Online_Marketing_adstock_mean','Weekly_Investment_SEM_mean','Weekly_Investment_affiliates_mean','Weekly_Sponsorship_adstock_mean','Weekly_Investment_Online_Marketing_mean','Weekly_Investment_Sponsorship_mean','Weekly_Investment_Digital_mean','Weekly_Investment_TV_mean','Weekly_Investment_Total_mean','Weekly_SEM_adstock_mean','Weekly_affiliates_adstock_mean','Weekly_Digital_adstock_mean'],axis=1,inplace = True)
#Determining VIF
Camera_vif = find_vif(Camera_X_train)
Camera_vif

,Features,VIF
0,units_sum,838475.67
8,s1_fact.order_payment_type_sum,389526.22
9,Prepaid_Flag_sum,113104.81
2,product_mrp_mean,2287.51
3,product_mrp_sum,1467.22
4,product_procurement_sla_mean,1383.57
1,sla_mean,1302.97
17,list_price,71.50
16,discount_off_percent,63.48
14,Max Temp (°C)_max,61.03


In [111]:
# Dropping columns with high VIF values 

Camera_X_train.drop(['units_sum','s1_fact.order_payment_type_sum','Prepaid_Flag_sum'],axis=1,inplace = True)
#Determining VIF
Camera_vif = find_vif(Camera_X_train)
Camera_vif

,Features,VIF
3,product_procurement_sla_mean,884.24
0,sla_mean,494.35
1,product_mrp_mean,279.99
11,Max Temp (°C)_max,57.68
14,list_price,30.90
13,discount_off_percent,27.52
2,product_mrp_sum,25.24
7,Weekly_TV_adstock_mean,20.07
12,Min Temp (°C)_min,17.92
15,shelf_price_infl,12.12


In [112]:
# Dropping columns with high VIF values 

Camera_X_train.drop(['sla_mean','product_mrp_mean','product_procurement_sla_mean'],axis=1,inplace = True)
#Determining VIF
Camera_vif = find_vif(Camera_X_train)
Camera_vif

,Features,VIF
8,Max Temp (°C)_max,39.18
0,product_mrp_sum,16.45
9,Min Temp (°C)_min,15.49
4,Weekly_TV_adstock_mean,15.21
10,discount_off_percent,14.23
11,list_price,13.13
12,shelf_price_infl,8.86
6,Rain_Flag,7.04
1,Weekly_Investment_Content_Marketing_mean,2.66
2,Weekly_Investment_Radio_mean,2.43


In [113]:
# Dropping columns with high VIF values 

Camera_X_train.drop(['Max Temp (°C)_max'],axis=1,inplace = True)
#Determining VIF
Camera_vif = find_vif(Camera_X_train)
Camera_vif

,Features,VIF
0,product_mrp_sum,16.15
9,discount_off_percent,14.15
10,list_price,11.24
11,shelf_price_infl,8.52
4,Weekly_TV_adstock_mean,7.09
6,Rain_Flag,6.88
8,Min Temp (°C)_min,5.01
1,Weekly_Investment_Content_Marketing_mean,2.58
2,Weekly_Investment_Radio_mean,2.37
3,Pay_Day_Flag,2.19


In [114]:
# Dropping columns with high VIF values 

Camera_X_train.drop(['list_price'],axis=1,inplace = True)
#Determining VIF
Camera_vif = find_vif(Camera_X_train)
Camera_vif

,Features,VIF
0,product_mrp_sum,13.14
10,shelf_price_infl,8.38
4,Weekly_TV_adstock_mean,6.92
9,discount_off_percent,6.92
6,Rain_Flag,6.60
8,Min Temp (°C)_min,3.96
1,Weekly_Investment_Content_Marketing_mean,2.57
2,Weekly_Investment_Radio_mean,2.19
3,Pay_Day_Flag,2.15
7,Snow_Flag,1.88


In [115]:
# Dropping columns with high VIF values 

Camera_X_train.drop(['product_mrp_sum','Weekly_TV_adstock_mean'],axis=1,inplace = True)
#Determining VIF
Camera_vif = find_vif(Camera_X_train)
Camera_vif

,Features,VIF
8,shelf_price_infl,6.31
7,discount_off_percent,5.15
4,Rain_Flag,2.90
6,Min Temp (°C)_min,2.81
0,Weekly_Investment_Content_Marketing_mean,2.28
1,Weekly_Investment_Radio_mean,1.82
2,Pay_Day_Flag,1.81
5,Snow_Flag,1.49
3,Sale_Flag,1.46


In [116]:
Camera_lr1,lr1_pred = returnOLS(Camera_y_train,Camera_X_train)
print(Camera_lr1.summary())

                            OLS Regression Results                            
Dep. Variable:                gmv_sum   R-squared:                       0.988
Model:                            OLS   Adj. R-squared:                  0.984
Method:                 Least Squares   F-statistic:                     246.9
Date:                Mon, 16 Sep 2019   Prob (F-statistic):           7.52e-23
Time:                        21:07:21   Log-Likelihood:                -499.75
No. Observations:                  36   AIC:                             1019.
Df Residuals:                      26   BIC:                             1035.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                                               coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------

In [117]:
Camera_X_train.drop(['Snow_Flag','Rain_Flag','Weekly_Investment_Radio_mean'],axis=1,inplace = True)
Camera_lr2,lr2_pred = returnOLS(Camera_y_train,Camera_X_train)
print(Camera_lr2.summary())

                            OLS Regression Results                            
Dep. Variable:                gmv_sum   R-squared:                       0.988
Model:                            OLS   Adj. R-squared:                  0.986
Method:                 Least Squares   F-statistic:                     401.8
Date:                Mon, 16 Sep 2019   Prob (F-statistic):           1.53e-26
Time:                        21:07:26   Log-Likelihood:                -500.25
No. Observations:                  36   AIC:                             1014.
Df Residuals:                      29   BIC:                             1026.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                                               coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------

In [118]:
Camera_X_train.drop(['Weekly_Investment_Content_Marketing_mean','Sale_Flag'],axis=1,inplace = True)
Camera_lr3,lr3_pred = returnOLS(Camera_y_train,Camera_X_train)
print(Camera_lr3.summary())

                            OLS Regression Results                            
Dep. Variable:                gmv_sum   R-squared:                       0.988
Model:                            OLS   Adj. R-squared:                  0.986
Method:                 Least Squares   F-statistic:                     634.1
Date:                Mon, 16 Sep 2019   Prob (F-statistic):           3.03e-29
Time:                        21:07:35   Log-Likelihood:                -500.53
No. Observations:                  36   AIC:                             1011.
Df Residuals:                      31   BIC:                             1019.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                -1.231e+05 

In [ ]:
Camera_y_test = Camera_test.pop('gmv_sum')
Camera_X_test = Camera_test[Camera_X_train.columns]

In [120]:

# Add a constant
Camera_X_test_lm = sm.add_constant(Camera_X_test)
# Create a first fitted model
y_pred = Camera_lr3.predict(Camera_X_test_lm)
Camera_rmse,Camera_r2score = rmse_r2score(Camera_y_test,y_pred)

In [121]:
print('R_2 ',Camera_r2score)

R_2  -5.521079774011215
